In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
df=pd.read_csv('/content/fmnist_small.csv')

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
df.shape

(6000, 785)

In [6]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
X_train = X_train/255.0
X_test = X_test/255.0

In [13]:
class data(Dataset):
  def __init__(self,features,lables):
    self.features=torch.tensor(features,dtype=torch.float32)
    self.lables=torch.tensor(lables,dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self,idx):
    return self.features[idx],self.lables[idx]


In [14]:
train_dataset = data(X_train, y_train)

In [15]:
test_dataset = data(X_test, y_test)

In [25]:
class model(nn.Module):
  def __init__(self,neurons_per_layer,num_layers,dropout,input_dim,output_dim):
    super().__init__()

    layers=[]

    for i in range(num_layers):
      layers.append(nn.Linear(input_dim,neurons_per_layer))
      layers.append(nn.BatchNorm1d(neurons_per_layer))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(dropout))
      input_dim=neurons_per_layer

    layers.append(nn.Linear(neurons_per_layer,output_dim))


    self.model=nn.Sequential(*layers)

  def forward(self,x):
    return self.model(x)


In [26]:
def objective(trail):
  num_hidden_layer=trail.suggest_int('num_hidden_layer',1,10,step=1)
  neurons_per_layer=trail.suggest_int('neurons_per_layer',16,256,step=8)
  dropout=trail.suggest_float('dropout',0.1,0.5,step=0.1)
  learning_rate=trail.suggest_float('learning_rate',1e-5,1e-1,log=True)
  epochs=trail.suggest_int('epochs',10,100,step=10)
  batch_size=trail.suggest_categorical('batch_size',[16, 32, 64, 128])
  optimizer_name=trail.suggest_categorical('optimizer',['adam','sgd'])
  input_dim=784
  output_dim=10

  net=model(neurons_per_layer,num_hidden_layer,dropout,input_dim,output_dim)
  criterion=nn.CrossEntropyLoss()

  if optimizer_name=='adam':
    optimizer=torch.optim.Adam(net.parameters(),lr=learning_rate)
  else:
    optimizer=torch.optim.SGD(net.parameters(),lr=learning_rate)

  train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
  test_loader=DataLoader(test_dataset,batch_size=batch_size,shuffle=False)


  for epoch in range(epochs):
    for features,labels in train_loader:
      outputs=net(features)
      loss=criterion(outputs,labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()


  net.eval()

  total=0
  correct=0

  with torch.no_grad():
    for features,labels in test_loader:
      outputs=net(features)
      _,predicted=torch.max(outputs.data,1)
      total+=labels.size(0)
      correct+=(predicted==labels).sum().item()

      accuracy=100*correct/total
  return accuracy

In [27]:
!pip install optuna

In [28]:
import optuna
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=10)

[I 2025-10-01 17:42:45,340] A new study created in memory with name: no-name-87d26e1c-3403-4bb2-b4fd-229ef6c295eb
[I 2025-10-01 17:43:08,792] Trial 0 finished with value: 41.666666666666664 and parameters: {'num_hidden_layer': 8, 'neurons_per_layer': 80, 'dropout': 0.4, 'learning_rate': 0.0006402089808339748, 'epochs': 40, 'batch_size': 128, 'optimizer': 'adam'}. Best is trial 0 with value: 41.666666666666664.
[I 2025-10-01 17:43:44,584] Trial 1 finished with value: 80.58333333333333 and parameters: {'num_hidden_layer': 5, 'neurons_per_layer': 64, 'dropout': 0.30000000000000004, 'learning_rate': 6.304973558200464e-05, 'epochs': 70, 'batch_size': 32, 'optimizer': 'adam'}. Best is trial 1 with value: 80.58333333333333.
[I 2025-10-01 17:43:58,121] Trial 2 finished with value: 78.58333333333333 and parameters: {'num_hidden_layer': 1, 'neurons_per_layer': 16, 'dropout': 0.2, 'learning_rate': 0.0037038816460804305, 'epochs': 50, 'batch_size': 16, 'optimizer': 'sgd'}. Best is trial 1 with val

In [29]:
study.best_value

83.83333333333333

In [31]:
study.best_params

{'num_hidden_layer': 2,
 'neurons_per_layer': 192,
 'dropout': 0.1,
 'learning_rate': 0.003012269240096696,
 'epochs': 50,
 'batch_size': 32,
 'optimizer': 'sgd'}